In [1]:
#importing the needed libraries
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_validate, KFold
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, f1_score, confusion_matrix, accuracy_score

In [2]:
#reading in the data we will use in the machine learning pipeline, with the top ten features of the method
df = pd.read_excel('5fs_impedance_training_set_10_final.xlsx')
df

,ImP_exp_diff_mean,ImP_exp_dur_sd,ImP_RRV_RMSSD,ImP_insp_flow_sd,ImP_ie_ratio_sd,ImP_duty_cycle_mean,ImP_insp_amp_sd,ImP_RSP_Symmetry_PeakTrough,ImP_exp_amp_sd,ImP_exp_dur_mean,Task_Label,Participant,Classification
0,0.118857,0.171499,382.434308,0.015663,0.048638,0.489158,0.050985,0.517348,0.046774,1.694000,12.0a,10785,0
1,0.225143,0.173505,390.361005,0.020811,0.069124,0.485452,0.030734,0.528718,0.031616,1.748000,12.0b,10785,0
2,0.236000,0.222699,434.669530,0.016278,0.050025,0.476758,0.031615,0.520548,0.026435,2.272000,12.0c,10785,0
3,0.250667,0.190379,403.425334,0.015714,0.071244,0.483507,0.029114,0.530733,0.025312,1.931429,12.0d,10785,0
4,0.140800,0.174310,483.103302,0.006162,0.124387,0.481444,0.043006,0.527149,0.028574,2.132000,12.0e,10785,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2569,0.240000,0.274684,318.223192,0.003850,0.265283,0.489451,0.008020,0.524840,0.014866,1.499600,85,98586,0
2570,0.708000,0.764264,1236.868088,0.072599,0.375168,0.427189,0.121972,0.511842,0.102499,2.301714,87,98586,1
2571,0.119556,0.122849,159.285907,0.004550,0.082769,0.499069,0.006324,0.543171,0.012527,1.433600,89,98586,0
2572,1.567500,0.980966,2899.130116,0.021556,0.546540,0.482030,0.047870,0.472444,0.051081,1.872000,91,98586,1


In [3]:
# extracting the features from the DataFrame
features = df.drop(columns=['Classification', 'Participant', 'Task_Label'])

# extracting the Classification (0/1) and participants from the dataframe
target = df['Classification'].values
participants = df['Participant'].values
task = df['Task_Label'].values

# extracting the feature names
feature_names = features.columns.tolist()

# converting the features, target, participants to numpy arrays
features_array = features.values
target_array = target

# creating the dictionary with the needed format to enter into the nested cross-validation
data_dict = {
    'data': features_array,
    'target': target_array,
    'feature_names': feature_names,
    'participants': participants,
    'task': task
}

data_dict

{'data': array([[1.18857100e-01, 1.71499300e-01, 3.82434308e+02, ...,
         5.17347900e-01, 4.67743000e-02, 1.69400000e+00],
        [2.25142900e-01, 1.73505000e-01, 3.90361005e+02, ...,
         5.28718100e-01, 3.16159000e-02, 1.74800000e+00],
        [2.36000000e-01, 2.22698600e-01, 4.34669530e+02, ...,
         5.20547800e-01, 2.64352000e-02, 2.27200000e+00],
        ...,
        [1.19555600e-01, 1.22848900e-01, 1.59285907e+02, ...,
         5.43171200e-01, 1.25266000e-02, 1.43360000e+00],
        [1.56750000e+00, 9.80965500e-01, 2.89913012e+03, ...,
         4.72444400e-01, 5.10806000e-02, 1.87200000e+00],
        [2.16500000e-01, 1.61567600e-01, 3.19002015e+02, ...,
         5.20748900e-01, 1.69911000e-02, 1.65511110e+00]]),
 'target': array([0, 0, 0, ..., 0, 1, 0]),
 'feature_names': ['ImP_exp_diff_mean',
  'ImP_exp_dur_sd',
  'ImP_RRV_RMSSD',
  'ImP_insp_flow_sd',
  'ImP_ie_ratio_sd',
  'ImP_duty_cycle_mean',
  'ImP_insp_amp_sd',
  'ImP_RSP_Symmetry_PeakTrough',
  'ImP_exp_am

In [4]:
#running the nested cross-validation for the Impedance (using VU-AMS 5fs) method with Random Forest
from sklearn.metrics import accuracy_score, precision_score, recall_score
import xgboost as xgb

# random seed for reproducibility
random_seed = 1
np.random.seed(random_seed)

X = data_dict['data']
Y = data_dict['target']
groups = data_dict['participants']

# setting up the parameter grid: the combinations of these will be tried out for ever outer loop's inner loop
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

rfc = RandomForestClassifier(random_state=random_seed, class_weight="balanced")

logo = LeaveOneGroupOut()


# defining the techniques to use for the inner and outer loops
inner_cv = KFold(n_splits=5, shuffle=True, random_state=1)
outer_cv = logo.split(X, Y, groups)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1_score': make_scorer(f1_score, average='macro'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, average='weighted')
}

# performing nested cross-validation
clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=inner_cv)
cv_dic = cross_validate(clf, X, Y, cv=outer_cv, scoring=scoring, return_estimator=True, return_train_score=False)
mean_acc_score = cv_dic['test_accuracy'].mean()
mean_prec_score = cv_dic['test_precision'].mean()
mean_rec_score = cv_dic['test_recall'].mean()
mean_f1_score = cv_dic['test_f1_score'].mean()
mean_roc_auc = cv_dic['test_roc_auc'].mean()  # Mean ROC AUC score


print('nested_accuracy_scores:   ', cv_dic['test_accuracy'])
print('mean score:            {0:.4f}'.format(mean_acc_score))

print('nested_precision_scores:   ', cv_dic['test_precision'])
print('mean score:            {0:.4f}'.format(mean_prec_score))

print('nested_recall_scores:   ', cv_dic['test_recall'])
print('mean score:            {0:.4f}'.format(mean_rec_score))

print('nested_f1_scores:   ', cv_dic['test_f1_score'])
print('mean score:            {0:.4f}'.format(mean_f1_score))

print('nested_roc_auc_scores:   ', cv_dic['test_roc_auc'])
print('mean score:            {0:.4f}'.format(mean_roc_auc))

nested_accuracy_scores:    [0.95454545 0.95454545 0.86363636 0.78787879 0.89393939 0.89393939
 0.92424242 0.90909091 0.90909091 0.65151515 0.75757576 0.81818182
 0.90909091 0.89393939 1.         0.92424242 0.86363636 0.83333333
 0.93939394 0.63636364 0.75757576 0.95454545 0.86363636 0.86363636
 0.87878788 0.78787879 0.86363636 0.87878788 0.86363636 0.8030303
 0.87878788 0.93939394 0.93939394 0.96969697 0.95454545 0.92424242
 0.83333333 0.74242424 0.87878788]
mean score:            0.8691
nested_precision_scores:    [0.95757576 0.95459141 0.86694493 0.79265734 0.90909091 0.90756587
 0.92981093 0.91223776 0.91831852 0.71337795 0.81616162 0.86246658
 0.91223776 0.89525499 1.         0.92398198 0.88797654 0.88571429
 0.94466403 0.67897727 0.77828283 0.95757576 0.87782767 0.87024087
 0.8885851  0.78787879 0.88797654 0.87878788 0.87782767 0.87223587
 0.90909091 0.93939394 0.94213287 0.97202797 0.95538803 0.92532151
 0.87052342 0.82724683 0.8885851 ]
mean score:            0.8866
nested_recal

In [5]:
sensitivity_scores = []
specificity_scores = []

# iteration over the outer folds to calculate average sensitivity and specificity
for i, (train_index, test_index) in enumerate(logo.split(X, Y, groups)):
    print(f"Fold {i + 1}:")

    # getting the trained estimator for the present fold
    estimator = cv_dic['estimator'][i]
    
    # predicting on the test set
    y_pred = estimator.predict(X[test_index])
    
    # creating confusion matrix
    tn, fp, fn, tp = confusion_matrix(Y[test_index], y_pred).ravel()
    
    # calculating the sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("=" * 30)
    
    # store the sensitivity and specificity scores
    sensitivity_scores.append(sensitivity)
    specificity_scores.append(specificity)

    
    # calculate and print the averages of the sensitivity and specificity scores previously calculated
average_sensitivity = np.mean(sensitivity_scores)
average_specificity = np.mean(specificity_scores)

print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)

Fold 1:
Sensitivity: 0.8750
Specificity: 1.0000
Fold 2:
Sensitivity: 0.9167
Specificity: 0.9762
Fold 3:
Sensitivity: 0.7083
Specificity: 0.9524
Fold 4:
Sensitivity: 0.7500
Specificity: 0.8095
Fold 5:
Sensitivity: 0.7083
Specificity: 1.0000
Fold 6:
Sensitivity: 0.9583
Specificity: 0.8571
Fold 7:
Sensitivity: 0.9583
Specificity: 0.9048
Fold 8:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 9:
Sensitivity: 0.9583
Specificity: 0.8810
Fold 10:
Sensitivity: 0.7917
Specificity: 0.5714
Fold 11:
Sensitivity: 0.9167
Specificity: 0.6667
Fold 12:
Sensitivity: 0.9583
Specificity: 0.7381
Fold 13:
Sensitivity: 0.9167
Specificity: 0.9048
Fold 14:
Sensitivity: 0.8750
Specificity: 0.9048
Fold 15:
Sensitivity: 1.0000
Specificity: 1.0000
Fold 16:
Sensitivity: 0.8750
Specificity: 0.9524
Fold 17:
Sensitivity: 0.9583
Specificity: 0.8095
Fold 18:
Sensitivity: 1.0000
Specificity: 0.7381
Fold 19:
Sensitivity: 0.8333
Specificity: 1.0000
Fold 20:
Sensitivity: 0.7083
Specificity: 0.5952
Fold 21:
Sensitivity: 0.7917


In [6]:
#print all performance scores that might be of interest
import scipy.stats

def calculate_stats(data):
    mean = np.mean(data)
    sd = np.std(data, ddof=1)  # using sample standard deviation (N-1)
    ci = scipy.stats.norm.interval(0.95, loc=mean, scale=sd/np.sqrt(len(data)))
    min_val = np.min(data)
    max_val = np.max(data)
    return mean, sd, ci, min_val, max_val

keys = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1_score', 'test_roc_auc', 'sensitivity_scores', 'specificity_scores']

for key in keys:
    if key in cv_dic:
        data = cv_dic[key]
    elif key == 'sensitivity_scores':
        data = sensitivity_scores
    elif key == 'specificity_scores':
        data = specificity_scores

    stats = calculate_stats(data)
    print(f"{key}: Mean={stats[0]}, SD={stats[1]}, 95% CI={stats[2]}, Min={stats[3]}, Max={stats[4]}")


test_accuracy: Mean=0.8690753690753689, SD=0.08155542748196605, 95% CI=(0.8434795721432616, 0.8946711660074761), Min=0.6363636363636364, Max=1.0
test_precision: Mean=0.8865778459148789, SD=0.06713501879706714, 95% CI=(0.865507828121852, 0.9076478637079057), Min=0.6789772727272727, Max=1.0
test_recall: Mean=0.8690753690753689, SD=0.08155542748196605, 95% CI=(0.8434795721432616, 0.8946711660074761), Min=0.6363636363636364, Max=1.0
test_f1_score: Mean=0.8633361389427632, SD=0.08189237936149622, 95% CI=(0.8376345912084024, 0.8890376866771239), Min=0.6309412861136998, Max=1.0
test_roc_auc: Mean=0.949239417989418, SD=0.05149475651874021, 95% CI=(0.9330780247919703, 0.9654008111868657), Min=0.7569444444444444, Max=1.0
sensitivity_scores: Mean=0.8963675213675214, SD=0.08754269784864233, 95% CI=(0.8688926470862463, 0.9238423956487966), Min=0.7083333333333334, Max=1.0
specificity_scores: Mean=0.8534798534798532, SD=0.11658332422095506, 95% CI=(0.8168907117812283, 0.8900689951784782), Min=0.57142

In [7]:
# IF this is the best-performing nested CV for the method: print best hyperparameters for each outer loop
for fold_idx, estimator in enumerate(cv_dic['estimator']):
    print(f'Fold {fold_idx + 1} - Best hyperparameters: {estimator.best_params_}')

Fold 1 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 2 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 3 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 4 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 100}
Fold 5 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 6 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 25}
Fold 7 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 150}
Fold 8 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
Fold 9 - Best hyperparameters: {'max_depth': 6, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 

In [8]:
# IF this is the best-performing nested CV for the method: print mode of hyperparameters

from collections import defaultdict, Counter

# initialize a dictionary to contain all hyperparameters and their values across loops
hyperparameters = defaultdict(list)

# extracting hyperparameters of each outer loop
for estimator in cv_dic['estimator']:
    for param, value in estimator.best_params_.items():
        hyperparameters[param].append(value)

# count the most frequent value for each hyperparameter
for param, values in hyperparameters.items():
    most_common_value, count = Counter(values).most_common(1)[0]  # Get the most frequent value
    print(f'Hyperparameter: {param} - Most frequent value: {most_common_value} (Count: {count})')


Hyperparameter: max_depth - Most frequent value: 6 (Count: 37)
Hyperparameter: max_features - Most frequent value: None (Count: 37)
Hyperparameter: max_leaf_nodes - Most frequent value: 9 (Count: 39)
Hyperparameter: n_estimators - Most frequent value: 25 (Count: 15)
